* Karpathy video: https://www.youtube.com/watch?v=kCc8FmEb1nY
* Karpathy repo: https://github.com/karpathy/nanoGPT
* Colab for video: https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing

# Setup:

In [1]:
data_source = "maestro"

if data_source == "shakespeare":
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
else:
    !wget -N https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
    !unzip -n -qq maestro-v3.0.0-midi.zip

--2023-11-25 19:53:22--  https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.215.207, 173.194.219.207, 64.233.185.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.215.207|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘maestro-v3.0.0-midi.zip’ not modified on server. Omitting download.



In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from glob import glob
import pathlib
import pretty_midi
from tqdm import tqdm
import numpy as np
import random 


In [3]:
use_wandb = True
if use_wandb:
    import wandb
    wandb.login()

wandb: Currently logged in as: drscotthawley. Use `wandb login --relogin` to force relogin


In [4]:
# Karpathy's # hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------


# Mine for Pitches
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 128 # 64
n_head = 8 # 4
n_layer = 4 * 4
dropout = 0.1


config = {
  'learning_rate': learning_rate,
  'batch_size': batch_size,
  'block_size': block_size,
  'n_embd': n_embd,
  'n_head': n_head,
  'n_layer': n_layer,
  'dropout': dropout
}

In [5]:
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

print('device =',device) 

device = cuda:1


# Dataset creation

In [6]:
notes_list = torch.load('maestro3_tensor_list.pt')  # load from previous computation
#notes_list = torch.load('rastro-120bpm_16th_tensor_list.pt')  # load from previous computation
len(notes_list)

1276

In [7]:
def tl_to_notes(tensor_list, shuffle=False, delimit=True):
  "list of tensors (of arbitrary length, for each song) converted to one big long tensor of notes all running togehter"
  if shuffle:random.shuffle(tensor_list)
  if delimit:
    delimiter = torch.zeros(3)  # use all zeros to show ends of songs
    tensor_list = [element for item in tensor_list for element in (item, delimiter)]
  return torch.vstack(tensor_list).type(torch.float32)  # return one big tensor of floats

all_notes = tl_to_notes(notes_list, shuffle=True) 
all_pitches = all_notes[:,0].type(torch.long)  # just the pitch info
all_notes.shape, all_pitches.shape

(torch.Size([7041440, 3]), torch.Size([7041440]))

Codebook creation

In [17]:
#chars = list(text)
# data here.
#raw_data = torch.tensor([ord(c) for c in chars],dtype=torch.int32).unsqueeze(1) # sadly torch has no char dtype
#raw_data = np.array(list(all_pitches))[:,np.newaxis] # but numpy does
raw_data = all_pitches.unsqueeze(-1)
print("raw_data.shape =",raw_data.shape)
n_codebooks = raw_data.shape[-1]
print("n_codebooks =",n_codebooks)
codebooks = []
for i in range(n_codebooks): 
    #cb_vals = sorted(set(raw_data[:,i]))
    cb_vals = raw_data.unique().sort()[0] if i>0 else torch.arange(128)  # use all possible pitches 
    print(f"cb {i}: cb_vals = {cb_vals}")
    codebooks.append({'encode':{k.item(): int(v) for v, k in enumerate(cb_vals)}, 
                      'decode':{int(v): k for v, k in enumerate(cb_vals)}})

vocab_sizes = [len(cb['encode']) for cb in codebooks]
print("vocab_sizes = ",vocab_sizes)
vocab_size = vocab_sizes[0]
vocab_size

raw_data.shape = torch.Size([7041440, 1])
n_codebooks = 1
cb 0: cb_vals = tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127])
vocab_sizes =  [128]


128

Encoder and decoder

In [18]:
encode = lambda s: [codebooks[0]['encode'][c.item()] for c in s]
decode = lambda l: [codebooks[0]['decode'][i] for i in l]
#test_str = 'hello!'
test_str = [torch.tensor(q) for q in [66,32,88,0,98]]
ilist = encode(test_str)
print("ilist = ",ilist)
ret_str = decode(ilist)
assert test_str==ret_str, f"Oops. test_str={test_str}, but ret_str={ret_str}"

ilist =  [66, 32, 88, 0, 98]


In [19]:
torch.manual_seed(1337)

# # here are all the unique characters that occur in this text
# chars = sorted(list(set(text)))
# vocab_size = len(chars)
# # create a mapping from characters to integers
# stoi = { ch:i for i,ch in enumerate(chars) }
# itos = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
#data = torch.tensor(encode(text), dtype=torch.long).unsqueeze(-1)  # make it like the music data
data = torch.tensor(encode(all_pitches), dtype=torch.long).unsqueeze(-1)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print("train_data.shape, val_data.shape =",train_data.shape, val_data.shape)


def augment_data(x,y): # db = datablock - a seqeunce , likely of length 1+block_size 
    x,y = x.clone(), y.clone()  # avoid in-place alterations of data
    pitch_shift = torch.randint(low=-6, high=6, size=(1,1,1), dtype=torch.long).item()
    x, y = [torch.clamp(q + pitch_shift, 0, vocab_size-1) for q in [x,y]] 
    return x,y


# data loading
def get_batch(split, debug=False):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = augment_data(x,y)
    x, y = x.to(device), y.to(device)
    if debug: print(f"get_batch: x.shape = {x.shape}, y.shape = {y.shape}")
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


train_data.shape, val_data.shape = torch.Size([6337296, 1]) torch.Size([704144, 1])


In [22]:
#%%timeit 
#x,y = get_batch('val')
# output: 1.55 ms ± 916 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

In [23]:
x,y = get_batch('train',debug=True) # first time is slow
print(f"B, T = {batch_size}, {block_size}")

get_batch: x.shape = torch.Size([64, 256, 1]), y.shape = torch.Size([64, 256, 1])
B, T = 64, 256


x is a sequence

In [24]:
x[0].T  # .T is to show it horizontal instead of vertical

tensor([[63, 59, 75, 71, 68, 59, 66, 56, 63, 56, 75, 66, 68, 71, 63, 59, 59, 66,
         56, 63, 68, 71, 75, 66, 59, 56, 63, 75, 71, 68, 35, 23, 28, 56, 59, 66,
         62, 50, 44, 67, 66, 62, 56, 71, 74, 78, 62, 66, 56, 59, 71, 74, 78, 81,
         66, 62, 59, 56, 59, 56, 62, 71, 83, 74, 78, 66, 81, 62, 66, 56, 59, 71,
         78, 74, 66, 56, 59, 62, 78, 71, 74, 66, 62, 59, 56, 78, 74, 71, 66, 59,
         62, 56, 66, 56, 62, 59, 78, 74, 76, 66, 62, 59, 56, 74, 78, 71, 35, 23,
         56, 60, 47, 66, 62, 40, 66, 60, 72, 80, 76, 56, 56, 60, 66, 62, 58, 80,
         76, 88, 84, 86, 56, 86, 72, 84, 80, 76, 66, 56, 60, 62, 66, 60, 56, 62,
         72, 80, 76, 66, 56, 60, 62, 80, 72, 76, 66, 56, 62, 60, 66, 62, 56, 76,
         80, 60, 72, 66, 56, 60, 62, 72, 76, 80, 81, 62, 66, 56, 60, 73, 85, 80,
         76, 64, 61, 56, 59, 35, 23, 74, 80, 52, 86, 42, 83, 62, 68, 64, 92, 80,
         58, 86, 80, 70, 74, 92, 86, 64, 88, 80, 74, 92, 64, 86, 70, 87, 90, 67,
         91, 79, 58, 86, 64,

y is x shifted back by one and including new data.
in this sense only y[:,-1] is the "next token" being predicted.

In [25]:
y[0].T

tensor([[59, 75, 71, 68, 59, 66, 56, 63, 56, 75, 66, 68, 71, 63, 59, 59, 66, 56,
         63, 68, 71, 75, 66, 59, 56, 63, 75, 71, 68, 35, 23, 28, 56, 59, 66, 62,
         50, 44, 67, 66, 62, 56, 71, 74, 78, 62, 66, 56, 59, 71, 74, 78, 81, 66,
         62, 59, 56, 59, 56, 62, 71, 83, 74, 78, 66, 81, 62, 66, 56, 59, 71, 78,
         74, 66, 56, 59, 62, 78, 71, 74, 66, 62, 59, 56, 78, 74, 71, 66, 59, 62,
         56, 66, 56, 62, 59, 78, 74, 76, 66, 62, 59, 56, 74, 78, 71, 35, 23, 56,
         60, 47, 66, 62, 40, 66, 60, 72, 80, 76, 56, 56, 60, 66, 62, 58, 80, 76,
         88, 84, 86, 56, 86, 72, 84, 80, 76, 66, 56, 60, 62, 66, 60, 56, 62, 72,
         80, 76, 66, 56, 60, 62, 80, 72, 76, 66, 56, 62, 60, 66, 62, 56, 76, 80,
         60, 72, 66, 56, 60, 62, 72, 76, 80, 81, 62, 66, 56, 60, 73, 85, 80, 76,
         64, 61, 56, 59, 35, 23, 74, 80, 52, 86, 42, 83, 62, 68, 64, 92, 80, 58,
         86, 80, 70, 74, 92, 86, 64, 88, 80, 74, 92, 64, 86, 70, 87, 90, 67, 91,
         79, 58, 86, 64, 90,

# Model definition

In [26]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_tables = nn.ModuleList([nn.Embedding(vocab_sizes[cb], n_embd) for cb in range(n_codebooks)])
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_heads = nn.ModuleList([nn.Linear(n_embd, vocab_sizes[cb]) for cb in range(n_codebooks)])


    def forward(self, idx, targets=None):
        B, T, CBS = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = 0
        for cb in range(CBS): # just sum codebook reps
            tok_emb = tok_emb + self.token_embedding_tables[cb](idx[:,:,cb]) 
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)

        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits_list = [self.lm_heads[cb](x) for cb in range(CBS)] # list of (B,T,vocab_sizes)

        if targets is None:
            loss = None
        else:
            lambdas = [1.0]*CBS
            targets = targets.view(B*T)
            for cb in range(CBS): 
                logits = logits_list[cb]
                B, T, C = logits.shape
                #print("logits.shape =",logits.shape,", targets.shape =",targets.shape)
                logits = logits.view(B*T, -1)
                loss = F.cross_entropy(logits, targets)

        return logits_list, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:, :]
            # get the predictions
            logits_list, loss = self(idx_cond)
            idx_next_list = []
            for cb in range(ind_cond.shape[-1]):
                # focus only on the last time step
                logits = logitslist[cb][:, -1] # becomes (B, C)
                # apply softmax to get probabilities
                probs = F.softmax(logits, dim=-1) # (B, C)
                # sample from the distribution
                idx_next_list.append(torch.multinomial(probs, num_samples=1)) # (B, 1)
            idx_next = torch.tensor(idx_next_list).to(idx.device)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



Instantiate and get ready to run

In [27]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

3.232128 M parameters


In [28]:
def save_checkpoint(step, model, optimizer,loss, name):
    name = name + f'_{step}.pt'
    print("Saving checkpoint to",name)
    torch.save({
            'step': step,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, name)

def load_checkpoint(name, model, optimizer):
    checkpoint = torch.load(name)
    model.load_state_dict(checkpoint['model_state_dict'])
    m = model.to(device)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    step = checkpoint['step']
    loss = checkpoint['loss']
    return step, m, optimizer, loss

# Do training

In [29]:
if use_wandb: wandb.init(project='musicbox_pitches')

In [30]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if use_wandb: wandb.log(losses | {'step':iter//eval_interval})

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.9983, val loss 4.9980
step 100: train loss 3.7565, val loss 3.7474
step 200: train loss 3.4475, val loss 3.4340
step 300: train loss 3.3041, val loss 3.2970
step 400: train loss 3.2468, val loss 3.2389


KeyboardInterrupt: 

In [ ]:
save_checkpoint(max_iters, model, optimizer,loss, f"karpathy_base_model")

In [ ]:
if use_wandb: wandb.finish()

Generate

In [ ]:

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


In [ ]:
vocab_size